### 딥러닝 실습 과제 2주차 - CNN을 활용하여 보드 전체를 입력받아 9칸 상태 예측

다음  세 가지 활동을 해봅시다.

01. **모델 설계**: CNN 기반 분류 모델 활용
02. **손실 함수 정의**: 각 칸에 대해 3-클래스 분류
03. **학습 및 평가**: 시각화 함수, 정확도 측정, confusion matrix 등

TTTDataset.zip을 불러와 문제에서 요하는 코드를 구현하세요.

💡 **데이터 구조**  
- **`image_black`** : 이미지 데이터  
- **`labels`** : 타겟 데이터  

In [ ]:
# vscode 작업을 위한 필수 패키지 설치
%pip install torch torchvision numpy matplotlib scikit-learn tqdm pillow

In [11]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import json
import glob
import os

#### 00. 클래스
정의한 클래스를 이용해 실행해 주세요.

In [12]:
class TTTDataset(Dataset):
    def __init__(self, image_paths, label_paths, transform=None):
        """
        틱택토 데이터셋을 PyTorch Dataset 형태로 변환.
        :param image_paths: 이미지 파일 경로 리스트
        :param label_paths: 레이블 JSON 파일 경로 리스트
        :param transform: 이미지 전처리 변환
        """
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.transform = transform
        self.data = self._load_data()


    def _load_data(self):
        """ 이미지 & 레이블 로드 """
        data = []
        for img_path, lbl_path in zip(self.image_paths, self.label_paths):
            # 이미지를 흑백(Grayscale)로 변환
            image = Image.open(img_path).convert("L")  # "RGB" 대신 "L" 사용

            # JSON 레이블 로드
            with open(lbl_path, 'r') as f:
                labels = json.load(f)

            # 레이블을 숫자로 변환 (O=1, X=-1, blank=0)
            label_tensor = torch.tensor(
                [1 if v == "O" else -1 if v == "X" else 0 for v in labels.values()],
                dtype=torch.float32
            )
            data.append((image, label_tensor))

        return data


    def __len__(self):
        """ 데이터셋 크기 반환 """
        return len(self.data)


    def __getitem__(self, idx):
        """ 데이터셋에서 idx 번째 샘플(이미지 & 레이블)을 가져오는 역할 """
        image, label = self.data[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [13]:
# 01. 이미지와 레이블 파일 경로 로드
image_dir = os.path.join("../TTTDataset", "image_black")
labels_dir = os.path.join("../TTTDataset", "labels")

# 모든 이미지와 라벨 파일의 경로 가져오기 (jpg, JPG 확장자 모두)
jpg_image_paths = glob.glob(os.path.join(image_dir, "*.jpg"))
JPG_image_paths = glob.glob(os.path.join(image_dir, "*.JPG"))
image_paths = sorted(jpg_image_paths + JPG_image_paths)
label_paths = sorted(glob.glob(os.path.join(labels_dir, "*.json")))

print(f"이미지 파일 수: {len(image_paths)}")
print(f"라벨 파일 수: {len(label_paths)}")
print(f"첫 번째 이미지 경로: {image_paths[0]}")
print(f"첫 번째 레이블 경로: {label_paths[0]}")
print()

# 02. 이미지 전처리를 위한 transform 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),               # 이미지 크기 조정
    transforms.ToTensor(),                       # PIL 이미지를 텐서로 변환 (0-255 → 0-1)
    transforms.Normalize(mean=[0.5], std=[0.5])  # 흑백 이미지 정규화 (-1 ~ 1)
])

# 전처리가 잘 되는지 확인하기 - 첫 번째 이미지에 적용
sample_image = Image.open(image_paths[0]).convert("L")
processed_image = transform(sample_image)
print(f"원본 이미지 크기: {sample_image.size}")
print(f"전처리 후 텐서 크기: {processed_image.shape}")
print(f"텐서 값 범위: {processed_image.min():.4f} ~ {processed_image.max():.4f}")

이미지 파일 수: 453
라벨 파일 수: 453
첫 번째 이미지 경로: ../TTTDataset/image_black/01.jpg
첫 번째 레이블 경로: ../TTTDataset/labels/01_labels.json

원본 이미지 크기: (512, 512)
전처리 후 텐서 크기: torch.Size([1, 224, 224])
텐서 값 범위: -1.0000 ~ 1.0000


In [14]:
# 03. 데이터셋 생성
from torch.utils.data import random_split

# 전체 데이터셋 생성
full_dataset = TTTDataset(image_paths, label_paths, transform=transform)

# 데이터 분할 비율 설정
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# 전체 데이터 개수
total_size = len(full_dataset)
train_size = int(total_size * train_ratio)
val_size = int(total_size * val_ratio)
test_size = total_size - train_size - val_size

# 데이터셋 분할하기
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, 
    [train_size, val_size, test_size]
)

# 분할된 데이터셋 정보 출력
print(f"전체 데이터셋 크기: {total_size}")
print(f"학습 데이터셋 크기: {len(train_dataset)}")
print(f"검증 데이터셋 크기: {len(val_dataset)}")
print(f"테스트 데이터셋 크기: {len(test_dataset)}")

전체 데이터셋 크기: 453
학습 데이터셋 크기: 317
검증 데이터셋 크기: 67
테스트 데이터셋 크기: 69


In [15]:
# 04. 데이터로더 생성
from torch.utils.data import DataLoader

# 배치 크기 설정
batch_size = 32

# 전체 데이터로더 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,  # 학습 데이터는 섞어서 사용
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,  # 검증 데이터는 섞지 않음
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,  # 테스트 데이터는 섞지 않음
)

# DataLoader 정보 출력
print(f"학습 데이터 배치 수: {len(train_loader)}")
print(f"검증 데이터 배치 수: {len(val_loader)}")
print(f"테스트 데이터 배치 수: {len(test_loader)}")

학습 데이터 배치 수: 10
검증 데이터 배치 수: 3
테스트 데이터 배치 수: 3


#### 01. 모델 설계: CNN 기반 분류 모델 활용
- 입력: 틱택토 보드 전체 이미지 (예: 128x128)
- 출력: 9개의 상태(각 칸마다 O, X, blank 중 하나)

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ImprovedTTTBoardCNN(nn.Module):
    def __init__(self):
        super(ImprovedTTTBoardCNN, self).__init__()
        
        # ResNet 스타일 구조와 더 깊은 네트워크
        # 입력: 1채널 224x224 그레이스케일 이미지
        
        # 초기 컨볼루션 레이어
        self.initial_conv = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )  # 출력: 64 x 56 x 56
        
        # ResNet 스타일 블록
        self.block1 = self._make_res_block(64, 128)  # 출력: 128 x 28 x 28
        self.block2 = self._make_res_block(128, 256)  # 출력: 256 x 14 x 14
        self.block3 = self._make_res_block(256, 512)  # 출력: 512 x 7 x 7
        
        # Global Average Pooling
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))  # 출력: 512 x 1 x 1
        
        # 공간 피라미드 풀링 (여러 스케일에서 특징 추출)
        self.spp = SpatialPyramidPooling([1, 2, 4])
        
        # 최종 분류 레이어
        self.classifier = nn.Sequential(
            nn.Linear(512 * (1 + 4 + 16), 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 9 * 3)  # 9개 셀, 각 셀마다 3가지 클래스
        )
        
        # 데이터 증강을 위한 자가주의(self-attention) 모듈
        self.attention = SelfAttention(512)
        
        # 가중치 초기화
        self._initialize_weights()
        
    def _make_res_block(self, in_channels, out_channels):
        """ResNet 스타일 블록 생성"""
        return nn.Sequential(
            # 첫 번째 컨볼루션 유닛
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            
            # 두 번째 컨볼루션 유닛
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            
            # 세 번째 컨볼루션 유닛 (잔차 연결)
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def _initialize_weights(self):
        """네트워크 가중치 초기화"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        # 초기 특징 추출
        x = self.initial_conv(x)
        
        # ResNet 스타일 블록 통과
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        
        # 자가주의 적용
        x = self.attention(x)
        
        # 공간 피라미드 풀링
        x = self.spp(x)
        
        # 분류
        x = self.classifier(x)
        
        # 출력을 9개 셀, 각 셀마다 3 클래스로 재구성
        return x.view(-1, 9, 3)


class SpatialPyramidPooling(nn.Module):
    """공간 피라미드 풀링 - 다양한 크기의 특징을 감지하는 데 도움"""
    def __init__(self, levels):
        super(SpatialPyramidPooling, self).__init__()
        self.levels = levels
        
    def forward(self, x):
        batch_size, channels, h, w = x.size()
        features = []
        
        for level in self.levels:
            # 각 레벨에서 pooling 수행
            pool = nn.AdaptiveMaxPool2d((level, level))
            pooled = pool(x).view(batch_size, channels, -1)
            features.append(pooled)
        
        # 모든 레벨 특징 연결
        x = torch.cat(features, dim=2)
        return x.view(batch_size, -1)


class SelfAttention(nn.Module):
    """자가주의 모듈 - 이미지의 중요한 부분에 집중"""
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels, in_channels//8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels//8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        batch_size, C, width, height = x.size()
        
        # 쿼리 생성
        proj_query = self.query_conv(x).view(batch_size, -1, width*height).permute(0, 2, 1)
        
        # 키 생성
        proj_key = self.key_conv(x).view(batch_size, -1, width*height)
        
        # 어텐션 맵 계산
        energy = torch.bmm(proj_query, proj_key)
        attention = F.softmax(energy, dim=2)
        
        # 값 생성
        proj_value = self.value_conv(x).view(batch_size, -1, width*height)
        
        # 어텐션 적용
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        out = out.view(batch_size, C, width, height)
        
        # 잔차 연결
        out = self.gamma * out + x
        return out


# 모델 초기화 및 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImprovedTTTBoardCNN().to(device)
print(f"Using device: {device}")
print(model)

Using device: cpu
ImprovedTTTBoardCNN(
  (initial_conv): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (block1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
  )
  (block2): Sequential(
    (0): Conv2d(

#### 02. 손실 함수 정의: 각 칸에 대해 3-클래스 분류
- 다중 클래스 분류이므로 CrossEntropyLoss 사용 가능
- 9개 칸을 각각 분류하는 방식으로 모델 구성

In [17]:
import torch
import torch.nn as nn

def ttt_loss_function(predictions, targets):
    
    # CrossEntropyLoss 정의
    loss_fn = nn.CrossEntropyLoss()
    
    # 레이블 변환: [-1(X), 0(blank), 1(O)] → [2, 0, 1]
    converted_targets = torch.zeros_like(targets, dtype=torch.long)
    converted_targets[targets == -1] = 2  # X → 2
    converted_targets[targets == 0] = 0   # blank → 0
    converted_targets[targets == 1] = 1   # O → 1
    
    # 9개 셀 각각에 대한 손실 계산
    total_loss = 0
    for i in range(9):
        cell_pred = predictions[:, i, :]       # i번째 셀 예측값
        cell_target = converted_targets[:, i]  # i번째 셀 실제값
        total_loss += loss_fn(cell_pred, cell_target)
    
    # 평균 손실 반환
    return total_loss / 9

# 최적화 함수 설정 - Adam
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

#### 03. 학습 및 평가: 시각화 함수, 정확도 측정, confusion matrix 등
- 학습 데이터로 모델을 훈련하고, 정확도 측정
- 예측이 잘 되는지 시각화하여 분석

In [18]:
def train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        # 훈련 단계
        train_loss, train_acc = evaluate(model, train_loader, criterion, optimizer, is_training=True)
        
        # 검증 단계
        val_loss, val_acc = evaluate(model, val_loader, criterion, is_training=False)
        
        # 현재 epoch 결과 출력
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"Train - Loss: {train_loss:.4f}, Accuracy: {train_acc:.2%}")
        print(f"Valid - Loss: {val_loss:.4f}, Accuracy: {val_acc:.2%}")
        print("-" * 50)


def evaluate(model, data_loader, criterion, optimizer=None, is_training=True):
    model.train() if is_training else model.eval()
    
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.set_grad_enabled(is_training):
        for images, targets in data_loader:
            images = images.to(device)
            targets = targets.to(device)
            
            # 학습 모드일 경우 그래디언트 초기화
            if is_training and optimizer:
                optimizer.zero_grad()
                
            # 순전파
            outputs = model(images)
            loss = criterion(outputs, targets)
            
            # 학습 모드일 경우 역전파 및 최적화
            if is_training and optimizer:
                loss.backward()
                optimizer.step()
            
            running_loss += loss.item()
            
            # 예측 및 정확도 계산
            pred_classes = torch.argmax(outputs, dim=2)
            converted_targets = targets.clone()
            converted_targets[targets == -1] = 2  # X를 2로 변환
            
            correct += (pred_classes == converted_targets).all(dim=1).sum().item()
            total += targets.size(0)
    
    return running_loss / len(data_loader), correct / total


# 모델 학습 실행
num_epochs = 10
train_model(
    model=model,
    train_loader=train_loader, 
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=ttt_loss_function,
    num_epochs=num_epochs
)

Epoch [1/10]
Train - Loss: 1.0883, Accuracy: 0.00%
Valid - Loss: 1.0947, Accuracy: 0.00%
--------------------------------------------------
Epoch [2/10]
Train - Loss: 1.0491, Accuracy: 0.32%
Valid - Loss: 1.1553, Accuracy: 0.00%
--------------------------------------------------
Epoch [3/10]
Train - Loss: 0.9636, Accuracy: 0.95%
Valid - Loss: 1.5179, Accuracy: 0.00%
--------------------------------------------------
Epoch [4/10]
Train - Loss: 0.8803, Accuracy: 2.21%
Valid - Loss: 1.9323, Accuracy: 0.00%
--------------------------------------------------
Epoch [5/10]
Train - Loss: 0.8020, Accuracy: 2.21%
Valid - Loss: 1.7228, Accuracy: 0.00%
--------------------------------------------------
Epoch [6/10]
Train - Loss: 0.6962, Accuracy: 7.26%
Valid - Loss: 1.1918, Accuracy: 7.46%
--------------------------------------------------
Epoch [7/10]
Train - Loss: 0.6182, Accuracy: 11.36%
Valid - Loss: 0.9554, Accuracy: 0.00%
--------------------------------------------------
Epoch [8/10]
Train 